In [1]:
import math
import pytorch_lightning as pl
import torch
import torch.nn as nn
from torch.nn import Linear
from torch.nn import functional as F
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from tokenizers import Tokenizer
from torch.nn.utils.rnn import pad_sequence
import string
import pdb
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
from functools import partial

In [2]:
def gen_trg_mask(length, device):
    return torch.triu(
        torch.ones(length, length, device=device) * float("-inf"), diagonal=1
    )

def create_padding_mask(tensor, pad_idx):
    padding_mask = (tensor == pad_idx).transpose(0, 1)

    return padding_mask


def masked_accuracy(y_true: torch.Tensor, y_pred: torch.Tensor, pad_idx):
    mask = y_true != pad_idx
    y_true = torch.masked_select(y_true, mask)
    y_pred = torch.masked_select(y_pred, mask)
    acc = (y_true == y_pred).double().mean()
    return acc


class PositionalEncoding(nn.Module):
    #  https://pytorch.org/tutorials/beginner/transformer_tutorial.html

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 256):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """

        x = x + self.pe[: x.size(0)]

        return self.dropout(x)


class TokenEmbedding(nn.Module):
    #  https://pytorch.org/tutorials/beginner/translation_transformer.html
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: torch.Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)


class Seq2Seq(pl.LightningModule):
    def __init__(
        self,
        out_vocab_size,
        pad_idx,
        tokenizer,
        channels=256,
        dropout=0.1
    ):
        super().__init__()
        self.pad_idx = pad_idx
        self.dropout = dropout
        self.out_vocab_size = out_vocab_size

        self.embeddings = TokenEmbedding(
            vocab_size=self.out_vocab_size, emb_size=channels
        )

        self.pos_encoder = PositionalEncoding(d_model=channels, dropout=dropout)

        self.transformer = torch.nn.Transformer(
            d_model=channels,
            nhead=4,
            num_encoder_layers=6,
            num_decoder_layers=6,
            dim_feedforward=1024,
            dropout=dropout,
        )

        self.linear = Linear(channels, out_vocab_size)

        self.do = nn.Dropout(p=self.dropout)
        self.tokenizer = tokenizer

    def init_weights(self) -> None:
        init_range = 0.1
        self.embeddings.weight.data.uniform_(-init_range, init_range)
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-init_range, init_range)

    def encode_src(self, src):
        #pdb.set_trace()
        src = src.permute(1, 0)
        src_pad_mask = create_padding_mask(src, self.pad_idx)
        src = self.embeddings(src)
        src = self.pos_encoder(src)
        src = self.transformer.encoder(src, src_key_padding_mask=src_pad_mask)
        #src = self.pos_encoder(src)
        return src

    def decode_trg(self, trg, memory):
        trg = trg.permute(1, 0)
        out_sequence_len, batch_size = trg.size(0), trg.size(1)
        trg_pad_mask = create_padding_mask(trg, self.pad_idx)
        trg = self.embeddings(trg)
        trg = self.pos_encoder(trg)
        trg_mask = gen_trg_mask(out_sequence_len, trg.device)
        out = self.transformer.decoder(
            tgt=trg, memory=memory, tgt_mask=trg_mask, tgt_key_padding_mask=trg_pad_mask
        )
        out = out.permute(1, 0, 2)
        out = self.linear(out)
        return out

    def forward(self, x):
        #pdb.set_trace()
        src, trg = x
        src = self.encode_src(src)
        out = self.decode_trg(trg=trg, memory=src)
        return out

    def training_step(self, batch, batch_idx):
        return self._step(batch, batch_idx, name="train")

    def on_validation_epoch_start(self):
        self.acc_sum = 0
        self.acc_num = 0
    
    def validation_step(self, batch, batch_idx):
        return self._step(batch, batch_idx, name="valid")

    def on_validation_epoch_end(self):
        avg_acc = (self.acc_sum / self.acc_num).item()
        self.log("val_avg_acc", avg_acc)
        print("Epoch, accuracy:", self.current_epoch, round(avg_acc, 4))
    
    def test_step(self, batch, batch_idx):
        return self._step(batch, batch_idx, name="test")

    def _step(self, batch, batch_idx, name="train"):
        src, trg = batch
        #pdb.set_trace()
        trg_in, trg_out = trg[:, :-1], trg[:, 1:]
        y_hat_orig = self((src, trg_in))
        y_hat = y_hat_orig.view(-1, y_hat_orig.size(2))
        y = trg_out.contiguous().view(-1)
        
        if batch_idx==0 and name=="valid":
            tb_logger = None
            for logger in self.trainer.loggers:
                if isinstance(logger, TensorBoardLogger):
                    tb_logger = logger.experiment
                    break
            
            _, this_pred = torch.max(y_hat_orig, 2)
            for i in range(16):                
                text_truth = self.tokenizer.decode(trg_out[i].cpu().numpy())
                text_corrupted = self.tokenizer.decode(src[i].cpu().numpy())
                text_pred = self.tokenizer.decode(this_pred[i].cpu().numpy())
                
                output = "Corrupted: {}<br>Restored:{}".format(text_corrupted, text_pred)
                tb_logger.add_text(f'Validation #{i}, target: {text_truth}', output, self.global_step)
                
        
        loss = F.cross_entropy(y_hat, y, ignore_index=self.pad_idx)
        _, predicted = torch.max(y_hat, 1)
        acc = masked_accuracy(y, predicted, pad_idx=self.pad_idx)

        #pdb.set_trace()
        
        self.log(f"{name}_loss", loss)
        self.log(f"{name}_acc", acc)
        
        self.acc_sum += acc
        self.acc_num += 1

        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=3e-5)

In [3]:
MAX_LEN = 256

def apply_perturbation_to_text(text):
    #There's a chance it'll be perfect
    if np.random.uniform() < 0.1:
        return text
    
    all_characters = string.ascii_letters + ' '
    words = text.split()
    if np.random.uniform() < 0.1 and len(words) > 1:
        #Get rid of word
        idx = np.random.randint(len(words))
        words.pop(idx)                
    
    #Change word endings
    for i in range(len(words)):
        if words[i].endswith("ium") and np.random.uniform() < 0.01:
            words[i] = words[i][:-3] + "ius"
        elif words[i].endswith("ius") and np.random.uniform() < 0.01:
            words[i] = words[i][:-3] + "ium"
        elif words[i].endswith("us") and np.random.uniform() < 0.01:
            words[i] = words[i][:-2] + "i"     
        elif words[i].endswith("nem") and np.random.uniform() < 0.01:
            words[i] = words[i][:-3] + "nus"
    
    curr_str = " ".join(words)
    new_str = ""
    
    for i in range(len(curr_str)):
        if np.random.uniform() < 0.01:
            new_str += np.random.choice(list(all_characters))
        else:
            new_str += curr_str[i]
            
        #There's a chance for a new character to pop up
        if np.random.uniform() < 0.01:
            new_str += np.random.choice(list(all_characters))
            
    return new_str

def generate_batch(data_batch, pad_idx):
    src, trg = [], []
    for (src_item, trg_item) in data_batch:
        src.append(src_item)
        trg.append(trg_item)
    src = pad_sequence(src, padding_value=pad_idx, batch_first=True)
    trg = pad_sequence(trg, padding_value=pad_idx, batch_first=True)
    return src, trg



class Dataset(torch.utils.data.Dataset):
    def __init__(self, samples, hf_tokenizer):
        self.samples = samples
        self.n_samples = len(self.samples)
        self.hf_tokenizer = hf_tokenizer

    def __len__(self):
        return self.n_samples
    
    def __getitem__(self, idx):
        text = self.samples[idx]
        perturbed_text = apply_perturbation_to_text(text)

        x = self.hf_tokenizer.encode(perturbed_text).ids
        y = self.hf_tokenizer.encode(text).ids
        #print(text, len(x), len(y))
        #assert(len(x) < MAX_LEN)
        #assert(len(y) < MAX_LEN)
        x = torch.tensor(x, dtype=torch.long)
        y = torch.tensor(y, dtype=torch.long)

        return x, y


In [4]:
def split_text(text):
    words = text.split()
    i = 0
    lines = []
    while True:
        num_words = np.random.randint(2, 30)
        lines.append(" ".join(words[i:i+num_words]))
        i += num_words
        if i >= len(words):
            break
            
    return lines

with open("decretum.txt") as f:
    data = f.read()
    lines = split_text(data)
    
with open("corpus_thomisticum.txt") as f:
    data = f.read()
    lines += split_text(data)
    
with open("misc_medieval.txt") as f:
    data = f.read()
    lines += split_text(data)        
    
tokenizer = Tokenizer.from_file("latin_tokenizer.json")    
train, val = train_test_split(lines, test_size=0.1, random_state=1337)
train_data = Dataset(samples=train, hf_tokenizer=tokenizer)
val_data = Dataset(samples=val, hf_tokenizer=tokenizer)

print("len(train_data)", len(train_data))
print("len(val_data)", len(val_data))

len(train_data) 704475
len(val_data) 78276


In [5]:
lines[200]

'iusserit sine dubitatione serviendum est? Sicut enim in potestatibus societatis humane maior potestas minori ad obediendum preponitur,'

In [6]:
for l in lines:
    if len(l) < 5: print(l)
    if len(l) > 250: print(l)

civium, testimonia populorum, honoratorum arbitrium, electio clericorum in ordinationibus sacerdotum constituantur. Et infra: Per pacem et quietem sacerdotes, qui futuri sunt, postulentur, teneatur subscriptio clericorum, honoratorum testimonium, ordines conventus
quemlibet ecclesiasticarum rerum amministratorem, ut, verbi gratia, prepositum, yconomum, vicedominum; defensoris nomine advocatum sive castaldum, et iudicem; in subiecto regule archipresbiterum, archidiaconum, canonicum, monachum vel quemlibet mancipatum ecclesiastico offitio.
PRESUMPTIONE, NEC TEMPORIS PRESCRIPTIONE MUTARI POTEST. Licet regulis contineatur antiquis, parrochias unicuique ecclesiae pristina dispositione deputatas nulla posse ratione convelli, ne per consuetudinem pessimam, exempli mali temeritate crescente, universalis
ECCLESIARUM NULLA DEBENT INPROBITATE CONVELLI. Privilegia ecclesiarum, sanctorum canonibus instituta, et venerabilis Nicene fixa decretis, nulla possunt STATUTA PRIORUM SUCCESSORES SERVARE OPOR

In [7]:
def generate_batch(data_batch, pad_idx):
    src, trg = [], []
    for (src_item, trg_item) in data_batch:
        src.append(src_item)
        trg.append(trg_item)
    src = pad_sequence(src, padding_value=pad_idx, batch_first=True)
    trg = pad_sequence(trg, padding_value=pad_idx, batch_first=True)
    return src, trg

In [ ]:
train_loader = DataLoader(
    train_data,
    batch_size=128,
    num_workers=4,
    shuffle=True,
    collate_fn=partial(generate_batch, pad_idx=tokenizer.token_to_id("[PAD]")),
)
val_loader = DataLoader(
    val_data,
    batch_size=128,
    num_workers=4,
    shuffle=False,
    collate_fn=partial(generate_batch, pad_idx=tokenizer.token_to_id("[PAD]")),
)

model = Seq2Seq(
    out_vocab_size=tokenizer.get_vocab_size(),
    pad_idx=tokenizer.token_to_id("[PAD]"),
    tokenizer=tokenizer,
    dropout=0.1
)

checkpoint_callback = ModelCheckpoint(
    monitor="valid_acc", mode="max", dirpath="./", filename="checker"
)

logger = TensorBoardLogger(
        save_dir="./",
        name="autocorrect_logs",
    )

trainer = pl.Trainer(
    max_epochs=2000,
    logger=logger,
    callbacks=[checkpoint_callback],
)


trainer.fit(model, train_loader, val_loader)

/home/stanley/anaconda3/lib/python3.9/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/stanley/anaconda3/lib/python3.9/site-packages/pytorch_lightning/callb

Sanity Checking: 0it [00:00, ?it/s]

Epoch, accuracy: 0 0.0002


/home/stanley/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Training: 0it [00:00, ?it/s]

...

Epoch, accuracy: 391 0.9874


Validation: 0it [00:00, ?it/s]

Epoch, accuracy: 392 0.9876


Validation: 0it [00:00, ?it/s]

Epoch, accuracy: 393 0.9876
